In [1]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2
from shared_utils.rt_utils import show_full_df

import utils
import update_vars

In [17]:
import importlib
importlib.reload(utils)
importlib.reload(update_vars)

<module 'update_vars' from '/home/jovyan/data-analyses/thruway_intercity_bus/update_vars.py'>

In [19]:
ridership = pd.read_excel(update_vars.RIDERSHIP_PATH)

# Create utils to get stop sequence and distance info for ridership charts

* currently blocked on warehouse issues for SJJPA feed, could maybe do CC on `gtfs_funnel` cached files...

In [2]:
analysis_date = update_vars.ANALYSIS_DATE

In [3]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

## New SJJPA San Joaquins feed

In [4]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [5]:
sanj = feeds.query('name.str.contains("Amtrak San J")')
sanj

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,regional_feed_type,type
77,2b538004ce62dd20b1c326745da7834f,2025-09-10,d9e1e77d0754b712fc608741ae3836f5,America/Los_Angeles,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,524ea6209600e9a2de34a02cf9068729,Amtrak San Joaquins Schedule,None,schedule


In [6]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=update_vars.shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [7]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=update_vars.trip_cols)

In [9]:
sanj_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=sanj.feed_key, stop_cols=update_vars.stop_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [10]:
sanj_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=sanj.feed_key, get_df=True,
                                      trip_df = sanj_trips, stop_time_cols=update_vars.stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [21]:
# sanj_shapes_trip_info = sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [14]:
# sanj_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

In [20]:
sanj_buses = sanj_trips.query('route_type == "3"')

In [24]:
sanj_trips.columns

Index(['trip_instance_key', 'trip_id', 'trip_short_name', 'base64_url',
       'feed_key', 'name', 'regional_feed_type', 'gtfs_dataset_key',
       'service_date', 'direction_id', 'block_id', 'route_key', 'route_id',
       'route_type', 'route_short_name', 'route_long_name', 'route_desc',
       'agency_id', 'network_id', 'shape_array_key', 'shape_id'],
      dtype='object')

In [27]:
bus_st = sanj_st.merge(sanj_buses[['trip_id', 'direction_id', 'route_id', 'shape_array_key', 'shape_id']], on='trip_id')

In [40]:
bus_stops = bus_st.stop_id.unique()

In [31]:
import numpy as np

In [60]:
get_ridership_unique_stops = lambda df: np.union1d(df.orig, df.dest)

In [61]:
rider_stops = get_ridership_unique_stops(ridership)

In [62]:
rider_stops

array(['ACA', 'ANA', 'ARC', 'ARN', 'ATA', 'BAR', 'BBS', 'BFD', 'BKY',
       'BUL', 'BUR', 'CBZ', 'CIC', 'CLM', 'CLV', 'CML', 'COC', 'COX',
       'CPC', 'CPN', 'CRM', 'DAV', 'DBP', 'EKA', 'EKG', 'ELE', 'EMY',
       'EPL', 'FFV', 'FIL', 'FMT', 'FNO', 'FRT', 'FTA', 'FUL', 'GAC',
       'GBV', 'GDL', 'GTA', 'GVB', 'HAY', 'HEA', 'HNF', 'HSU', 'IDO',
       'IRV', 'KGC', 'LAX', 'LCA', 'LEG', 'LIV', 'LMC', 'LOD', 'LPS',
       'LQT', 'LTM', 'LTV', 'LVN', 'LVS', 'LVT', 'MCD', 'MDP', 'MDR',
       'MMK', 'MOD', 'MOJ', 'MPK', 'MRM', 'MRP', 'MRV', 'MRY', 'MTZ',
       'MYA', 'MYH', 'MYM', 'MYT', 'MYU', 'NAP', 'NHL', 'OAC', 'OKJ',
       'OLT', 'ONA', 'ORV', 'OSD', 'OXN', 'PAS', 'PCV', 'PDC', 'PLS',
       'PRB', 'PSP', 'PSS', 'PTC', 'RBF', 'RDR', 'RIC', 'RIV', 'RLN',
       'RLP', 'RNO', 'RPC', 'RSV', 'SAC', 'SAN', 'SAP', 'SAT', 'SBA',
       'SBU', 'SCC', 'SCS', 'SCZ', 'SFC', 'SIM', 'SJC', 'SKN', 'SKT',
       'SLH', 'SLO', 'SLP', 'SLT', 'SLV', 'SMN', 'SNA', 'SNB', 'SNC',
       'SNS', 'SOL',

In [41]:
bus_stops

array(['bSAC', 'bCPC', 'bPCV', 'bSLT', 'bSLH', 'bFRT', 'bSJC', 'bSKN',
       'bSKT', 'bTRA', 'bLIV', 'bDBP', 'bMRV', 'bORV', 'bCIC', 'bSCS',
       'bDAV', 'bLOD', 'bRSV', 'bRLN', 'bARN', 'bTRU', 'bRNO', 'bMRM',
       'bMRP', 'bMDP', 'bELP', 'bYOC', 'bYOV', 'bYOS', 'bMCD', 'bRDD',
       'bRBF', 'bEKG', 'bATA', 'bPRB', 'bLMC', 'bHNF', 'bVIS', 'bSAT',
       'bGVB', 'bSLO', 'bSLP', 'bOAH', 'bFSH', 'bYOM', 'bYOH', 'bFNO',
       'bCGD', 'bWTS', 'bUKH', 'bCLV', 'bSRC', 'bPTC', 'bVAL', 'bMTZ',
       'bHSU', 'bARC', 'bEKA', 'bFTA', 'bGBV', 'bLEG', 'bLTV', 'bHEA',
       'bRPC', 'bNAP', 'bSPX', 'bCOX', 'bEMY', 'bSFC', 'bYOT', 'bYOW',
       'bYOF', 'bMMK', 'bJNL', 'bLVN', 'bLOS', 'bGLD', 'bBFD', 'bSCZ',
       'bSVY', 'bSNB', 'bRIV', 'bONA', 'bCLM', 'bPAS', 'bLCA', 'bSBU',
       'bSBA', 'bCPN', 'bVEC', 'bOXN', 'bSAP', 'bFIL', 'bTEH', 'bMOJ',
       'bBAR', 'bBBS', 'bLVT', 'bLAS', 'bSMN', 'bWES', 'bVNC', 'bBUR',
       'bNHL', 'bSAN', 'bSOL', 'bOSD', 'bIRV', 'bSNA', 'bANA', 'bFUL',
      

In [44]:
sanj_bus_to_rider_stops = dict(zip(bus_stops, [x[1:] for x in bus_stops]))

In [67]:
pd.Series(sanj_bus_to_rider_stops.values()).isin(pd.Series(rider_stops)).value_counts()

True     102
False     11
dtype: int64

In [69]:
gtfs_no_rider_data = pd.Series(sanj_bus_to_rider_stops.values())[~pd.Series(sanj_bus_to_rider_stops.values()).isin(pd.Series(rider_stops))]
gtfs_no_rider_data #  OK -- all either served by RABA/YARTS or outside CA...

26    ELP
31    RDD
43    OAH
44    FSH
45    YOM
46    YOH
48    CGD
74    JNL
76    LOS
77    GLD
99    LAS
dtype: object